In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", None)
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [3]:
data = pd.read_csv("learn.csv")

In [4]:
del data["Unnamed: 0"]
data.loanapply_insert_time = pd.to_datetime(data.loanapply_insert_time)
data.insert_time = pd.to_datetime(data.insert_time)
data.company_enter_month = pd.to_datetime(data.company_enter_month)

"1. user들의 “비합리성” 척도: 승인된 application에 대해서 product를 이자율별로 나열시켜놓은 후에 가장 이자율을 적게 하는 경우의 이자율과 실제 선택된 상품을 기준으로 한 이자율 사이의 비율\n",
    "2. application별 product 수 히스토그램\n",
    "3. 모든 product의 loan_limit 히스토그램\n",
    "4. credit_score과 loan_rate의 히스토그램, credit_score과 desired_amount의 히스토그램 (신용점수가 낮을수록 돈을 더 많이 빌리고 싶어하나? 혹은 덜 빌리고 싶어하나?)\n",
    "5. credit_score과 평균 승인률 (application별 승인률)의 히스토그램\n",
    "6. user별로 application을 승인한 Session의 길이 / 평균 Session의 길이\n",
    "7. user별로 Session당 본인인증한 비율 vs. 전체 application 중 승인률 scatter-plot\n",
    "8. desired_amount와 application별 평균 승인률 히스토그램, desired_amount / yearly_income과 application별 평균 승인률 히스토그램 (갚을 능력이 얼마나 있는가)\n",
    "9. application에 대하여 product들의 평균 이자율과 승인여부 히스토그램\n",
    "10. 날짜별로 application의 전체 개수 line-graph\""

In [5]:
c = data.groupby("application_id").max()

In [6]:
k = c.loc[c.is_applied == 1]

#10
fig = go.Figure(data=[go.Histogram(x=k.loanapply_insert_time,
                                   xbins=dict(
                                       start='2022-03-01',
                                       end='2022-05-31',
                                              size='D7'),
                                   autobinx=False)])
fig.show()

In [10]:
#2
data.loc[:,"count"] = np.ones(len(data))
c = data.groupby("application_id").sum()

px.histogram(x = c["count"])

#3
x = np.log10(data.loan_limit+1)
plt.hist(x, bins = np.arange(40)/4)
plt.savefig("image.png")

#4-1
px.histogram(x = data.credit_score, y = data.loan_rate, histfunc = "avg", nbins = 90)

#4-2
px.histogram(x = data.credit_score, y = data.desired_amount, histfunc = "avg", nbins = 90)

In [33]:
#5
c = data.groupby("application_id").max()

In [34]:
c.reset_index(inplace = True, drop = True)

In [35]:
d = c.groupby("user_id").mean()

px.histogram(x = d.credit_score, y = d.is_applied, histfunc = "avg", nbins = 90)

#8
px.histogram(x = np.log10(d.desired_amount+1), y = d.is_applied, histfunc = "avg", nbins = 100)

In [25]:
c = data.groupby("application_id").max()
d = data.groupby("application_id").mean()

px.histogram(x = d.loan_rate, y = c.is_applied, histfunc = "sum", nbins = 100)